In [3]:
import pandas as pd

# 1. LOAD THE DATASETS
# Ensure the filenames match exactly what you have on your computer
file_b03 = "C:\\Users\\shrey\\Downloads\\WB Data\\DDW-1900B-03.xlsx" 
file_c08 = "C:\\Users\\shrey\\Downloads\\WB Data\\DDW-1900C-08.xlsx"

print("Processing datasets... Please wait.")

# 2. PROCESS WORKER DATA (B-03)
# Skipping 6 rows as Census Excel files contain metadata in the first few rows
df_b = pd.read_excel(file_b03, skiprows=6, header=None)

# Map the columns: 1=State, 2=Dist, 3=Area, 4=TRU, 5=EduLevel, 8=Pop_F, 11=MainW_F, 14=MargW_F
df_b = df_b[[1, 2, 3, 4, 5, 8, 11, 14]]
df_b.columns = ['StateCode', 'DistCode', 'AreaName', 'TRU', 'EduLevel', 'Pop_F', 'MainW_F', 'MargW_F']

# Get overall totals for each district (where EduLevel is 'Total')
workers = df_b[df_b['EduLevel'] == 'Total'].copy()
workers['Total_Female_Workers'] = pd.to_numeric(workers['MainW_F']) + pd.to_numeric(workers['MargW_F'])
workers = workers[['StateCode', 'DistCode', 'AreaName', 'TRU', 'Pop_F', 'Total_Female_Workers']]

# 3. PROCESS LITERACY & AGE DATA (C-08)
df_c = pd.read_excel(file_c08, skiprows=6, header=None)

# Map the columns: 1=State, 2=Dist, 4=TRU, 5=AgeGroup, 8=Total_F, 14=Lit_F
df_c = df_c[[1, 2, 4, 5, 8, 14]]
df_c.columns = ['StateCode', 'DistCode', 'TRU', 'AgeGroup', 'Pop_F_C08', 'Lit_F']

# Extract Literates from 'All ages' row
lit = df_c[df_c['AgeGroup'] == 'All ages'][['DistCode', 'TRU', 'Lit_F', 'Pop_F_C08']].copy()

# Extract 0-6 age group to adjust the literacy rate denominator (Population 7+)
pop06 = df_c[df_c['AgeGroup'] == '0-6'][['DistCode', 'TRU', 'Pop_F_C08']].copy()
pop06.rename(columns={'Pop_F_C08': 'Pop06_F'}, inplace=True)

# 4. MERGE AND CALCULATE
final = pd.merge(workers, lit, on=['DistCode', 'TRU'])
final = pd.merge(final, pop06, on=['DistCode', 'TRU'])

# Convert to numeric to avoid calculation errors
for col in ['Pop_F', 'Total_Female_Workers', 'Lit_F', 'Pop_F_C08', 'Pop06_F']:
    final[col] = pd.to_numeric(final[col], errors='coerce')

# CALCULATE RATES
# Literacy Rate = (Literates / Population aged 7+) * 100
final['Female_Literacy_Rate'] = (final['Lit_F'] / (final['Pop_F_C08'] - final['Pop06_F'])) * 100
# FWPR = (Total Workers / Total Population) * 100
final['FWPR'] = (final['Total_Female_Workers'] / final['Pop_F']) * 100

# 5. FILTER FOR WEST BENGAL DISTRICTS
# State Code 19 is West Bengal. We exclude DistCode 0 (State Total) to focus on Districts.
wb_final = final[(final['StateCode'] == 19) & (final['DistCode'] != 0)].copy()

# Clean AreaName and Format
wb_final['AreaName'] = wb_final['AreaName'].str.replace('District - ', '', regex=False)
wb_final = wb_final[['AreaName', 'TRU', 'Female_Literacy_Rate', 'FWPR', 'Pop_F', 'Total_Female_Workers', 'Lit_F']]
wb_final = wb_final.round(2)

# VIEW THE DATASET
print("\n--- FINAL DATASET PREVIEW (First 10 rows) ---")
print(wb_final.head(10))

# SAVE THE DATASET
wb_final.to_excel('WB_Female_Literacy_Work_Analysis.xlsx', index=False)
print("\nSuccess! The final dataset has been saved as 'WB_Female_Literacy_Work_Analysis.xlsx'")

Processing datasets... Please wait.

--- FINAL DATASET PREVIEW (First 10 rows) ---
          AreaName    TRU  Female_Literacy_Rate   FWPR      Pop_F  \
3       Darjiling   Total                 73.33  22.44   909564.0   
4       Darjiling   Rural                 66.59  26.01   551895.0   
5       Darjiling   Urban                 83.48  16.93   357669.0   
6      Jalpaiguri   Total                 66.23  22.31  1889782.0   
7      Jalpaiguri   Rural                 61.82  25.05  1375209.0   
8      Jalpaiguri   Urban                 77.75  14.97   514573.0   
9      Koch Bihar   Total                 68.49  20.67  1367544.0   
10     Koch Bihar   Rural                 66.47  21.43  1224736.0   
11     Koch Bihar   Urban                 85.00  14.18   142808.0   
12  Uttar Dinajpur  Total                 52.17  18.96  1456068.0   

    Total_Female_Workers      Lit_F  
3               204140.0   597912.0  
4               143571.0   326042.0  
5                60569.0   271870.0  
6    